# Kubernetes Pods

In [ ]:
# Cleanup

kubectl get all
#kubectl delete po/2-containers-in-a-pod
#kubectl delete deploy k8s-demo
#kubectl delete service k8s-demo-service

We've seen already how we can interrogate our cluster server using the kubectl tool, obtaining information about objects such as some system pods, in the system.

Now let's create our own pods using kubectl.

Normally we would not create pods directly using command-line arguments, as we will do here.

We would create objects in a repeatable, declarative way from yaml files, but we will do that later.

The below ```kubectl run``` command will create 2 pods (replicas), each running just 1 container from the k8s-demo image.

Note that we can specify labels on any kubernetes objects, using the -l option followd by key=value pairs, e.g.
    -l "key1=val1,key2=val2"
    
These labels are extremely useful allowing us to select a group of objects using a label selector.

Labels might be used to mark components of the cluster as dev, test or production for example.

Or they might be used to signal hardware specifics of a node, to distinguish physical nodes with SSD disks, or GPU which could then be prioritized for particular types of processing which might be disk i/o or CPU intensive.

k8sdemo is a small Docker image of 6MBy on the Docker hub [HERE](https://hub.docker.com/r/mjbright/k8s-demo/tags/)

In [ ]:
echo; echo "-- initial kubectl get all"
kubectl get all

echo; echo "-- kubectl run k8s0demo"
kubectl run k8s-demo -l "app=k8s-demo" --image=mjbright/k8s-demo:1 --replicas=2 --port=8080

echo; echo "-- second kubectl get all"
kubectl get all

In the above output we see that the only running resource initially was an internal service.

Once we perform '''kubectl run'' this spawns a **deployment** called *k8s-demo*, which creates a corrsponding **replicaset** which in turn spawns 2 **pods**.

As we performed the '''kubectl get all''' immediately after the run, we see the status before the pod containers have been created ... in fact there is a delay whilst the images are downloaded (for a first run) and pods instantiated.

Re-running '''kubectl get all''' shows the pods as *running* now.

In [ ]:
kubectl get all

... with more information, using option '*-o wide*'

In [ ]:
kubectl get all -o wide

## Creating resources using yaml specifications

### Obtaining yaml as output

Using the option '-o yaml' to any 'kubectl get' command we can obtain a yaml definition, so instead of

In [ ]:
kubectl get deploy/k8s-demo -o wide

we get ...

In [ ]:
kubectl get deploy/k8s-demo -o yaml

In [ ]:
cat examples/3.pod_example.yaml

In [ ]:
kubectl get all

In [ ]:
#kubectl delete po/2-containers-in-a-pod
#kubectl get pods

In [ ]:
kubectl create -f examples/3.pod_example.yaml

In [ ]:
kubectl get all

In [ ]:
kubectl get all

In [ ]:
kubectl describe po/2-containers-in-a-pod

In [ ]:
kubectl exec -it 2-containers-in-a-pod -n default echo hello world

In [ ]:
kubectl logs 2-containers-in-a-pod shell | head -3

In [ ]:
kubectl delete po/2-containers-in-a-pod

In [ ]:
kubectl get pods

## What happens if a pod dies?

In [ ]:
kubectl get pods -o wide
kubectl scale --replicas=3 rs/xxxx
kubectl get pods -o wide

In [ ]:
kubectl delete pod newpod

In [ ]:
kubectl get pods -o wide

In [ ]:
kubectl scale --replicas=3 rs/xxxx

## What happens if a node dies?

Do we really want to try this?

## Accessing our application

To access our application pods we should expose them as a service.
This would have the advantage of being able to address a virtual ip address for our service as pods themselves come and go (so we don't need to determine their address), and also allows load balancing between pods.

### Accessing our application - directly accessing Pods

But there are other more basic ways that we will investigate first.

If we perform '''kubectl get pods -o wide''' we will see the cluster ip of our pods, enabling us to access directly to the pods.

In [ ]:
kubectl get pods -o wide

In [ ]:
POD1=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $1;}')
POD2=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $1;}')
POD1_IP=$(kubectl get pods -o wide --no-headers=true | head -1 | awk '{print $6;}')
POD2_IP=$(kubectl get pods -o wide --no-headers=true | tail -1 | awk '{print $6;}')

echo POD1=$POD1 POD1_IP=$POD1_IP
echo POD2=$POD2 POD2_IP=$POD2_IP

Let's look at the yaml information for POD1:

In [ ]:
kubectl get pod $POD1 -o yaml

So now we can access to the pods directly on their port 8080:

In [ ]:
curl http://$POD1_IP:8080

In [ ]:
curl http://$POD2_IP:8080

# Kube-proxy

## TODO: Accessing our application - accessing via kube-proxy

<hr/>